In [ ]:
from datetime import datetime

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter

from gfatpy.atmo import ecmwf, atmo
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.utils import generate_synthetic_signal
from gfatpy.lidar.retrieval.helper import RetrievalHelper
from gfatpy.lidar.plot.quicklook import quicklook_xarray
from gfatpy.utils import calibration, utils

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
calibration.split_continous_measurements()

### Load lidar data

In [ ]:
RS_FL = r"../tests/datos/MULHACEN/1a/2021/07/05/*Prs*.nc"
DC_FL = r"../tests/datos/MULHACEN/1a/2021/07/05/*Pdc*.nc"

lidar = preprocess(
    rs_fl=RS_FL,
    dc_fl=DC_FL,
    channels=["532xpa"],
    deadtime_flag=False,
    save_bg=True
)

In [ ]:
lidar.bg_532xpa.values

### Get temperature and pressure profiles

In [ ]:
atmo_data = ecmwf.get_ecmwf_day(datetime(2022, 7, 5), lidar.time, lidar.range)

In [ ]:
plt.pcolormesh(lidar.time, lidar.range, atmo_data["temperature"].T)
plt.xticks(rotation=45)
plt.xlabel("Range [m]")
plt.ylabel("Time")

bar = plt.colorbar()
bar.set_label("Temperature[K]")

plt.show()

In [ ]:
atmo_2d = calibration.molecular_properties_2d(
    "2022-07-05",
    wavelength=532,
    heights=lidar.range,
    times=lidar.time
)

In [ ]:
plt.pcolormesh(lidar.time, lidar.range, atmo_2d["attenuated_molecular_beta"].T)
plt.xticks(rotation=45)
plt.xlabel("Range [m]")
plt.ylabel("Time")

bar = plt.colorbar()
bar.set_label(r"Attenuated molecular beta ($\beta_{att}$) [$m^{-1}$]")

plt.show()

### Visualization

In [ ]:
def normalize_to(height: float, profile: np.ndarray):
    return profile / profile.sel(range=height, method="nearest")

In [ ]:
rcs = lidar.signal_532xpa[0] * lidar.range**2

In [ ]:
h = 5000

plt.title("Rayleigh Fit")
plt.plot(lidar.range, normalize_to(h, atmo_2d["attenuated_molecular_beta"][0]), c = 'k')
plt.plot(lidar.range, normalize_to(h, rcs), alpha = 0.6, lw = 0.5)

plt.yscale('log')

In [ ]:
quicklook_xarray(lidar.signal_532xpa, is_rcs = False)

### Free atmosphere algorithm

As seen in quicklook, dataset should be split within 4 regions with some space between them. Last one will be used since is the largest.

In [ ]:
meas_groups = split_continous_measurements(lidar.time.values)
%matplotlib inline

In [ ]:
group4 = lidar.signal_532xpa.loc[meas_groups[3]]

In [ ]:
quicklook_xarray(group4, is_rcs = False)

In [ ]:
temporal_mean = xr.apply_ufunc(
    utils.moving_average,
    group4.load(),
    kwargs={"window_size": 15}
)

In [ ]:
quicklook_xarray(
    temporal_mean,
    is_rcs = False
)

In [ ]:
plt.plot(group4[150] * temporal_mean.range**2)
plt.plot(temporal_mean[150] * temporal_mean.range**2)
plt.plot(savgol_filter(group4[150] * temporal_mean.range**2, 15, 3))

In [ ]:
atmo_2d_cropped = atmo_2d["attenuated_molecular_beta"].loc[meas_groups[3]]

### Single profile comparison

In [ ]:
plt.plot(atmo_2d_cropped[0]*14000, label="molecular backscatter")
plt.plot(group4[0], lw = 0.4, alpha = 0.3, label = "single profile")
plt.plot(temporal_mean[0], lw = 0.5, label = "time window mean")
plt.plot(savgol_filter(temporal_mean[0], 15, 3), lw = 0.5, c = "k", label = "Savgol filter")

plt.legend()
plt.yscale('log')

plt.show()

In [ ]:
a = calibration.iterative_fitting(
    savgol_filter(group4[0],15, 3),
    atmo_2d_cropped[0],
    window_size_bins=40
)

In [ ]:
860*7.5

In [ ]:
group4[0]

In [ ]:
atmo_2d_cropped

In [ ]:
1500 * 7.5

In [ ]:
temporal_mean[0]

In [ ]:
calibration.iterative_fitting(temporal_mean[0], atmo_2d_cropped)